TO VOXELS:

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import h5py 
from scipy.ndimage import binary_dilation #Q-score
from scipy.spatial.distance import cdist #Hausdorff Distance
from scipy.stats import wasserstein_distance #Wasserstein Distance
from scipy.ndimage import sobel #Gradient Magnitude Similarity Deviation

In [126]:
with h5py.File("train_point_clouds.h5", "r") as f:
    #all groups and datasets
    print("Keys in the HDF5 file:")
    print(list(f.keys())) 

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'train_point_clouds.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [3]:
with h5py.File("train_point_clouds.h5", "r") as f:
    if isinstance(f['0'], h5py.Group):
        print("Keys in group '0':")
        print(list(f['0'].keys()))
    else:
        print("'0' is not a group, it may be a dataset")


Keys in group '0':
['img', 'normals', 'points']


In [4]:
# Load the train dataset
with h5py.File("train_point_clouds.h5", "r") as f:
    points_data_0 = f['0']['points'][:]  #load 3D points data for sample '0'
    points_data_1 = f['1']['points'][:]  #load 3D points data for sample '1'
    # print("Points data shape:", points_data_0.shape)
    # print("Points data contents:", points_data_0)


In [5]:
print("samlpe 0")
print("Points data shape:", points_data_0.shape)
print("Points data contents:", points_data_0)
print('\n')
print("samlpe 1")
print("Points data shape:", points_data_1.shape)
print("Points data contents:", points_data_1)

samlpe 0
Points data shape: (25700, 3)
Points data contents: [[ 0.15       -0.1094358   0.47305447]
 [ 0.15       -0.0969358   0.47305447]
 [ 0.15       -0.0844358   0.47305447]
 ...
 [ 0.15       -0.1094358  -0.50194553]
 [ 0.15       -0.1094358  -0.51444553]
 [ 0.15       -0.1094358  -0.52694553]]


samlpe 1
Points data shape: (26200, 3)
Points data contents: [[ 0.15        0.02270992  0.52146947]
 [ 0.15        0.03520992  0.52146947]
 [ 0.15        0.04770992  0.52146947]
 ...
 [ 0.15       -0.02729008 -0.45353053]
 [ 0.15       -0.02729008 -0.46603053]
 [ 0.15       -0.02729008 -0.47853053]]


In [6]:
def point_cloud_to_voxel_grid(point_cloud, grid_size=28):
    """
    Converts a point cloud to a voxel grid.
    
    Parameters:
    - point_cloud: (num_points, 3) numpy array of point coordinates.
    - grid_size: The size of the voxel grid (e.g., 28 for 28x28x28 grid).
    
    Returns:
    - voxel_grid: A 3D numpy array of shape (grid_size, grid_size, grid_size) with values true or false.
    """
    # Normalize point cloud to fit in a [0, 1] cube
    normalized_cloud = (point_cloud - point_cloud.min(axis=0)) / (point_cloud.ptp(axis=0)) #ptp = range of values (max - min)
    
    # Scale to grid size
    scaled_points = (normalized_cloud * (grid_size - 1)).astype(int)
    
    # Create an empty voxel grid
    voxel_grid = np.zeros((grid_size, grid_size, grid_size), dtype=bool)
    
    # Fill the voxel grid
    voxel_grid[scaled_points[:, 0], scaled_points[:, 1], scaled_points[:, 2]] = True
    
    return voxel_grid


In [ ]:
# def point_cloud_to_voxel_grid(point_cloud, grid_size=28):
#     # Normalize point cloud to fit in a [0, 1] cube
#     normalized_cloud = (point_cloud - point_cloud.min(axis=0)) / (point_cloud.ptp(axis=0))
    
#     # Scale to grid size
#     scaled_points = (normalized_cloud * (grid_size - 1)).astype(int)
    
#     # Create an empty voxel grid
#     voxel_grid = np.zeros((grid_size, grid_size, grid_size), dtype=bool)
    
#     # Fill the voxel grid
#     voxel_grid[scaled_points[:, 0], scaled_points[:, 1], scaled_points[:, 2]] = True
    
#     return voxel_grid

# # Convert the points data to a voxel grid
# #voxel_grid = point_cloud_to_voxel_grid(points_data)
# #print("Voxel grid shape:", voxel_grid.shape)


In [8]:
voxel_0 = point_cloud_to_voxel_grid(points_data_0)
voxel_1 = point_cloud_to_voxel_grid(points_data_1)
print('voxel_0 shape:', voxel_0.shape)
print('voxel_1 shape:', voxel_1.shape)

voxel_0 shape: (28, 28, 28)
voxel_1 shape: (28, 28, 28)


In [9]:
np.save("voxel_grid_sample_0.npy", voxel_0)
np.save("voxel_grid_sample_1.npy", voxel_1)

## If you have voxels saved run from HERE 

In [4]:
voxel_0 = np.load("voxel_grid_sample_0.npy")
voxel_1 = np.load("voxel_grid_sample_1.npy")


In [5]:
voxel_0

array([[[ True,  True,  True, ..., False, False, False],
        [ True,  True,  True, ..., False, False, False],
        [ True,  True,  True, ..., False, False, False],
        ...,
        [False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True]],

       [[ True,  True,  True, ..., False, False, False],
        [ True, False,  True, ..., False, False, False],
        [ True, False,  True, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False,  True],
        [False, False, False, ..., False, False,  True],
        [False, False, False, ...,  True,  True,  True]],

       [[ True,  True,  True, ..., False, False, False],
        [ True, False,  True, ..., False, False, False],
        [ True, False,  True, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False,  True],
        [False, False, False, ..., False, Fal

In [6]:
voxel_1

array([[[False,  True,  True, ..., False, False, False],
        [ True,  True,  True, ..., False, False, False],
        [ True,  True,  True, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False,  True,  True, ..., False, False, False],
        [ True,  True, False, ..., False, False, False],
        [ True, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False,  True,  True, ..., False, False, False],
        [ True,  True, False, ..., False, False, False],
        [ True, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, Fal

In [ ]:
import numpy as np
import plotly.graph_objects as go

def plot_voxel(voxel_data, title="Voxel Grid"):
    # Get the indices of the active voxels
    x, y, z = voxel_data.nonzero()

    # Calculate distances from the center of the voxel grid
    center = np.array(voxel_data.shape) / 2
    distances = np.sqrt((x - center[0])**2 + (y - center[1])**2 + (z - center[2])**2)
    
    # Normalize distances for coloring (0 to 1 range)
    max_distance = np.max(distances)
    normalized_distances = distances / max_distance if max_distance > 0 else distances

    colors = normalized_distances  

    # Create the 3D scatter plot
    fig = go.Figure(data=[go.Scatter3d(
        x=x,
        y=y,
        z=z,
        mode='markers',
        marker=dict(size=3, symbol= 'square', color=colors, colorscale='Viridis', showscale=True)  # You can change colorscale
    )])
    
    # Update layout
    fig.update_layout(title=title, scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    ))
    
    fig.show()


In [45]:
plot_voxel(voxel_0, title="Voxel Grid Sample 0")
plot_voxel(voxel_1, title="Voxel Grid Sample 1")

In [10]:
# Inverted voxel 0
inv_voxel_0 = np.logical_not(voxel_0)

# Comparing i.e. similarity measures

## XOR Similarity Metric

The XOR similarity metric focuses on the fraction of matching voxels between two voxel grids by normalizing the XOR count to the total number of voxels. This measure assesses how similar two voxel grids are, with values ranging from 0 to 1.

- **XOR Similarity = 1**: The grids are identical in terms of occupied and unoccupied voxel positions.
- **XOR Similarity = 0**: The grids have completely opposite voxel configurations.

#### Definition

The XOR similarity metric can be defined as:

$$
\text{XOR Similarity}(A, B) = 1 - \frac{\text{XOR Count}}{\text{Total Voxels}}
$$

where the XOR count is calculated as:

$$
\text{XOR Count} = \sum_{i} (A_i \oplus B_i)
$$

Here, \( A_i \) and \( B_i \) denote the values of voxels in grids \( A \) and \( B \) respectively, and \( \oplus \) represents the XOR operation.

The total number of voxels is given by:

$$
\text{Total Voxels} = |A| = |B|
$$

where \( |A| \) and \( |B| \) denote the total number of voxels in grids \( A \) and \( B \).

#### Time complexity O(n)

#### Properties

1. **Range**: The XOR similarity metric ranges from 0 to 1, where 1 indicates perfect similarity and 0 indicates complete dissimilarity.
2. **Symmetry**
3. **Sensitivity to Differences**: The metric is sensitive to differences in both occupied and unoccupied voxel positions, making it suitable for applications requiring precise voxel comparisons.
4. **Computational Efficiency** O(n)

In [11]:
def xor_similarity(voxel1, voxel2):
    """
    Calculate the XOR similarity between two voxel arrays.

    The XOR similarity is defined as 1 minus the ratio of the number of differing elements
    (computed using the logical XOR operation) to the total number of elements.

    Parameters:
    voxel1 (np.ndarray): The first voxel array.
    voxel2 (np.ndarray): The second voxel array.

    Returns:
    float: The XOR similarity between the two voxel arrays, ranging from 0 to 1.
    """
    xor_count = np.logical_xor(voxel1, voxel2).sum()
    total_voxels = voxel1.size
    return 1 - xor_count / total_voxels


In [12]:
print("XOR similarity score between sample 0 and sample 0:", xor_similarity(voxel_0, voxel_0))
print("XOR similarity score between sample 0 and sample inv0:", xor_similarity(voxel_0, inv_voxel_0))
print("XOR similarity score between sample 1 and sample 1:", xor_similarity(voxel_1, voxel_1))
print("XOR similarity score between sample 0 and sample 1:", xor_similarity(voxel_0, voxel_1))
print("XOR similarity score between sample 1 and sample 0:", xor_similarity(voxel_1, voxel_0))

XOR similarity score between sample 0 and sample 0: 1.0
XOR similarity score between sample 0 and sample inv0: 0.0
XOR similarity score between sample 1 and sample 1: 1.0
XOR similarity score between sample 0 and sample 1: 0.7042182944606414
XOR similarity score between sample 1 and sample 0: 0.7042182944606414


## Hausdorff Distance 
Measure focusing on the maximum distance between the nearest points in each set.
- Hausdorff Distance = 0: The grids are identical in terms of occupied voxel positions.<br>
- Hausdorff Distance > 0: Some voxels in one grid are not directly aligned with voxels in the other grid.

#### Definition


$$
d_H(A, B) = \max\{ h(A, B), h(B, A) \}
$$


$$
h(A, B) = \max_{a \in A} \min_{b \in B} d(a, b)
$$

d(a, b) denotes the distance between points a and b.

$$ 
d(a, b) = \sqrt{(x_2 - x_1)^2 + (y_2 - y_1)^2 + (z_2 - z_1)^2} 
$$

#### Time complexity O(k1 x k2) where 
k1- number of occupied voxels in first grid<br>
k2- number of occupied voxels in 2nd grid


#### Properties

1. **Symmetry**
2. **Non-negativity**
3. **Identity of indiscernibles**: If \( d_H(A, B) = 0 \), then the sets \( A \) and \( B \) are equivalent in the sense that they contain the same points.
4. **Triangle Inequality**

In [13]:
def hausdorff_distance(voxel1, voxel2):
    """
    Calculate the Hausdorff distance between two 3D voxel grids.

    The Hausdorff distance is the maximum distance of a set to the nearest point in the other set.
    It measures how far the two voxel grids are from each other.

    Parameters:
    voxel1 (numpy.ndarray): A 3D numpy array representing the first voxel grid.
    voxel2 (numpy.ndarray): A 3D numpy array representing the second voxel grid.

    Returns:
    float: The Hausdorff distance between the two voxel grids.
    """
    # Get coordinates of occupied voxels in both grids
    coords1 = np.array(np.nonzero(voxel1)).T
    coords2 = np.array(np.nonzero(voxel2)).T

    # Compute distances from each point in voxel1 to the closest point in voxel2
    dists_1_to_2 = cdist(coords1, coords2, 'euclidean').min(axis=1)
    dists_2_to_1 = cdist(coords2, coords1, 'euclidean').min(axis=1)

    # Hausdorff distance is the max of these minimum distances
    hausdorff_dist = max(dists_1_to_2.max(), dists_2_to_1.max())
    return hausdorff_dist

In [14]:
print("Hausdorff distance between sample 0 and sample 0:", hausdorff_distance(voxel_0, voxel_0))
print("Hausdorff distance between sample 0 and sample inv0:", hausdorff_distance(voxel_0, inv_voxel_0))
print("Hausdorff distance between sample 1 and sample 1:", hausdorff_distance(voxel_1, voxel_1))
print("Hausdorff distance between sample 0 and sample 1:", hausdorff_distance(voxel_0, voxel_1))
print("Hausdorff distance between sample 1 and sample 0:", hausdorff_distance(voxel_1, voxel_0))

Hausdorff distance between sample 0 and sample 0: 0.0
Hausdorff distance between sample 0 and sample inv0: 8.48528137423857
Hausdorff distance between sample 1 and sample 1: 0.0
Hausdorff distance between sample 0 and sample 1: 8.246211251235321
Hausdorff distance between sample 1 and sample 0: 8.246211251235321


## Total Variation Distance (TVD)

A measure that quantifies the difference between two probability distributions represented by voxel grids. It is calculated as the normalized absolute difference between the corresponding voxels in the two grids.

- **Total Variation Distance = 0**: The grids are identical, meaning the corresponding voxel values are the same in both grids.<br>
- **Total Variation Distance > 0**: There are differences between the corresponding voxel values in the two grids.

#### Definition

The Total Variation Distance is defined as:

$$
\text{TVD}(A, B) = \frac{1}{2} \sum_{i} |A_i - B_i|
$$

We are using normalization so its [0,1], so the used formula is:

$$
   \text{TVD} = \frac{1}{2} \cdot \frac{\sum_{i} |A_i - B_i|}{\text{size of the grid}}
$$

Where \( A_i \) and \( B_i \) represent the values of the corresponding voxels in grids \( A \) and \( B \).

#### Time Complexity O(n)


where n is the total number of voxels in the voxel grids.

#### Properties

1. **Range**: The Total Variation Distance ranges from 0 to 1 (or 0 to 0.5 when normalized), where 0 indicates identical distributions and higher values indicate greater divergence.
2. **Symmetry**
3. **Non-negativity**
4. **Identity of indiscernibles**: If TVD(A, B) = 0 , then the voxel grids \( A \) and \( B \) are equivalent in terms of their voxel values.
5. **Triangle Inequality**

In [15]:
def total_variation_distance(voxel1, voxel2):
    """
    Calculate the Total Variation Distance (TVD) between two voxel arrays.

    The Total Variation Distance is a measure of the difference between two 
    probability distributions. It is used to measure the difference between two voxel arrays.

    Parameters:
    voxel1 (np.ndarray): The first voxel array (3D).
    voxel2 (np.ndarray): The second voxel array (3D).

    Returns:
    float: The Total Variation Distance between the two voxel arrays.
    """
    # Compute absolute difference and normalize
    abs_difference = np.abs(voxel1.astype(float) - voxel2.astype(float))
    tvd = np.sum(abs_difference) / (2 * voxel1.size)
    return tvd

In [16]:
print("Total Variation Distance between sample 0 and sample 0:", total_variation_distance(voxel_0, voxel_0))
print("Total Variation Distance between sample 1 and sample 1:", total_variation_distance(voxel_1, voxel_1))
print("Total Variation Distance between sample 0 and sample 1:",  total_variation_distance(voxel_0, voxel_1))
print("Total Variation Distance between sample 1 and sample 0:", total_variation_distance(voxel_1, voxel_0))

Total Variation Distance between sample 0 and sample 0: 0.0
Total Variation Distance between sample 1 and sample 1: 0.0
Total Variation Distance between sample 0 and sample 1: 0.1478908527696793
Total Variation Distance between sample 1 and sample 0: 0.1478908527696793


## 

## Wasserstein Distance

A measure that quantifies the difference between two probability distributions represented by voxel grids. The Wasserstein distance (also known as the Earth Mover's Distance) takes into account the distance that needs to be "moved" to transform one distribution into another.

- **Wasserstein Distance = 0**: The grids are identical, meaning the distributions represented by the voxel values are the same in both grids.<br>
- **Wasserstein Distance > 0**: There are differences between the distributions represented by the corresponding voxel values in the two grids.

#### Definition

The Wasserstein Distance between two probability distributions \( P \) and \( Q \) is defined as:

$$
W(P, Q) = \inf_{\gamma \in \Gamma(P, Q)} \int_{X \times Y} c(x, y) d\gamma(x, y)
$$

Where:
- \( \Gamma(P, Q) \) is the set of all possible couplings of \( P \) and \( Q \),
- \( c(x, y) \) is the cost function representing the distance between points \( x \) and \( y \).

#### Time Complexity O(n log n)


#### Properties

1. **Range**: The Wasserstein Distance is always non-negative and ranges from 0 to infinity, where 0 indicates identical distributions.
2. **Symmetry**
3. **Non-negativity**
4. **Identity of indiscernibles**: If \( W(P, Q) = 0 \), then the distributions \( P \) and \( Q \) are equivalent in terms of their voxel values.
5. **Triangle Inequality**


In [17]:
def wasserstein_dst_voxel(voxel1, voxel2):
    """
    Compute the Wasserstein distance between two voxel grids.
    The function flattens the 3D voxel grids into 1D arrays and then computes the 
    Wasserstein distance between these arrays.
    Parameters:
    voxel1 (numpy.ndarray): The first voxel grid.
    voxel2 (numpy.ndarray): The second voxel grid.
    Returns:
    float: The Wasserstein distance between the two voxel grids.
    """
    # Flatten the voxel grids to 1D arrays
    voxel1_flat = voxel1.ravel().astype(float)
    voxel2_flat = voxel2.ravel().astype(float)
    
    # Compute Wasserstein distance
    wasserstein_dist = wasserstein_distance(voxel1_flat, voxel2_flat)
    return wasserstein_dist

In [18]:
print("Wasserstein distance between sample 0 and sample 0:", wasserstein_dst_voxel(voxel_0, voxel_0))
print("Wasserstein distance between sample 1 and sample 1:", wasserstein_dst_voxel(voxel_1, voxel_1))
print("Wasserstein distance between sample 0 and sample 1:", wasserstein_dst_voxel(voxel_0, voxel_1))
print("Wasserstein distance between sample 1 and sample 0:", wasserstein_dst_voxel(voxel_1, voxel_0))

Wasserstein distance between sample 0 and sample 0: 0.0
Wasserstein distance between sample 1 and sample 1: 0.0
Wasserstein distance between sample 0 and sample 1: 0.02619351311953355
Wasserstein distance between sample 1 and sample 0: 0.02619351311953355


## Gradient Magnitude Similarity Deviation (GMSD)

Measure based on voxels gradient magnitudes. It quantifies how similar the structures within the voxel grids are by analyzing the gradient information, which highlights changes in intensity.
Gradient Magnitude is a measure of the rate of change of intensity or value in an image or a function at a given point.

- **GMSD = 1**: The grids are identical in terms of their gradient magnitudes.<br>
- **GMSD < 1**: There are differences in the gradient magnitudes between the two grids, indicating dissimilarity.

#### Definition

The GMSD is defined as:

$$
\text{GMSD}(A, B) = 1 - \frac{1}{N} \sum_{i} \left| \frac{G_A(i) - G_B(i)}{G_A(i) + G_B(i) + \epsilon} \right|
$$

Where:
- \( G_A(i) \) and \( G_B(i) \) are the gradient magnitudes at voxel \( i \) for grids \( A \) and \( B \).
- \( N \) is the total number of voxels.
- epsilon is a small constant added to prevent division by zero.

### Time Complexity O(n)

### Properties

1. **Range**: The GMSD ranges from 0 to 1, where 1 indicates identical gradient magnitudes and lower values indicate greater divergence.
2. **Symmetry**
3. **Non-negativity**
4. **Identity of indiscernibles**: If GMSD(A, B) = 1 , then the voxel grids \( A \) and \( B \) are equivalent in terms of their gradient magnitudes.


In [19]:
def gradient_magnitude_similarity_deviation(voxel1, voxel2):
    """
    Compute the Gradient Magnitude Similarity Deviation (GMSD) between two voxel grids.
    Parameters:
    voxel1 (ndarray): The first 3D voxel grid.
    voxel2 (ndarray): The second 3D voxel grid.
    Returns:
    float: The GMSD value, which is a measure of the similarity between the gradient magnitudes of the two voxel grids.
           A value closer to 1 indicates higher similarity, while a value closer to 0 indicates lower similarity.
    Notes:
    - The function computes the gradient magnitude of each voxel grid using the Sobel operator along each axis.
    - The absolute difference between the gradient magnitudes of the two voxel grids is then computed.
    - The GMSD is calculated as 1 minus the mean of the normalized absolute differences.
    - A small constant (1e-6) is added to the denominator to avoid division by zero.
    """
    # Compute the gradient magnitude of each voxel grid
    gradient_magnitude1 = np.sqrt(sum(sobel(voxel1, axis=i)**2 for i in range(3)))
    gradient_magnitude2 = np.sqrt(sum(sobel(voxel2, axis=i)**2 for i in range(3)))
    
    # Compute the absolute difference of the gradient magnitudes
    gmsd = 1- np.mean(np.abs((gradient_magnitude1 - gradient_magnitude2) / (gradient_magnitude1 + gradient_magnitude2 + 1e-6)))
    return gmsd

In [20]:
print("Gradient Magnitude Similarity Deviation between sample 0 and sample 0:", gradient_magnitude_similarity_deviation(voxel_0, voxel_0))
print("Gradient Magnitude Similarity Deviation between sample 1 and sample 1:", gradient_magnitude_similarity_deviation(voxel_1, voxel_1))
print("Gradient Magnitude Similarity Deviation between sample 0 and sample 1:", gradient_magnitude_similarity_deviation(voxel_0, voxel_1))
print("Gradient Magnitude Similarity Deviation between sample 1 and sample 0:", gradient_magnitude_similarity_deviation(voxel_1, voxel_0))
print("Gradient Magnitude Similarity Deviation between sample 0 and sample inv0:", gradient_magnitude_similarity_deviation(voxel_0, inv_voxel_0))


Gradient Magnitude Similarity Deviation between sample 0 and sample 0: 1.0
Gradient Magnitude Similarity Deviation between sample 1 and sample 1: 1.0
Gradient Magnitude Similarity Deviation between sample 0 and sample 1: 0.51025390625
Gradient Magnitude Similarity Deviation between sample 1 and sample 0: 0.51025390625
Gradient Magnitude Similarity Deviation between sample 0 and sample inv0: 1.0
